In [15]:
!pip install selenium
!pip install webdriver-manager

In [18]:
#import

import os
import requests
import time
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# Get the api key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

#create an object of OpenAI
openai = OpenAI()


headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url, headless=True, chrome_binary=None, wait_seconds=10):
        """
        Create this Website object from the given url using Selenium WebDriver.
        Uses webdriver-manager to fetch a compatible chromedriver automatically.
        Parameters:
          - url: target URL
          - headless: run chrome headless (True/False)
          - chrome_binary: optional path to chrome/chromium binary (if not in PATH)
          - wait_seconds: timeout for waiting page load/dynamic content
        """
        self.url = url
        options = Options()

        # headless or visible browser
        if headless:
            options.add_argument("--headless=new")  # use new headless flag where supported
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")  # helpful in containers
        options.add_argument("--disable-gpu")
        # some sites detect automation; these flags may help
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        # If you need to point to a custom Chrome/Chromium binary:
        if chrome_binary:
            options.binary_location = chrome_binary

        # Use webdriver-manager to download/manage chromedriver automatically
        service = Service(ChromeDriverManager().install())

        driver = webdriver.Chrome(service=service, options=options)
        try:
            driver.get(url)

            # Use WebDriverWait to let dynamic JS content load (better than sleep)
            try:
                WebDriverWait(driver, wait_seconds).until(
                    lambda d: d.execute_script("return document.readyState === 'complete'")
                )
            except Exception:
                # fallback: short sleep if readyState didn't hit complete in time
                time.sleep(2)

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # Title
            self.title = soup.title.string if soup.title else "No title found"

            # Remove irrelevant tags inside body if body exists
            body = soup.body or soup
            for irrelevant in body(["script", "style", "img", "input"]):
                irrelevant.decompose()

            self.text = body.get_text(separator="\n", strip=True)

        finally:
            driver.quit()



system_prompt = """You are a sarcastic website-parsing agent whose job is to produce a spoof/parody of a target website.
Behavior:
  - When given a URL, fetch and parse the page (assume access to tools like Selenium/Playwright and BeautifulSoup).
  - Preserve the site's structure: headings, subheadings, paragraphs, lists, and major sections.
  - Rewrite all visible copy in a clearly sarcastic, mocking, or humorous tone while preserving the original intent and structure so the spoof is recognizable.
  - Keep formatting (Markdown or HTML-like headings and lists) so the output can be rendered as a parody webpage.
  - Emphasize and exaggerate marketing fluff, UI oddities, and obvious clichés. Use witty, ironic, or deadpan phrasing.
Safety & Limits:
  - Do not produce content that is defamatory, reveals private personal data, or incites harassment. Jokes should target tone/marketing/design, not private individuals.
  - Avoid reproducing long verbatim copyrighted text; instead, paraphrase and transform content clearly into a parody.
  - If the page requires interactive steps (logins, paywalls, or dynamic user-only content), note the limitation and spoof using the visible public content only.
Output format:
  - Return a single spoofed document preserving headings and lists, suitable for rendering as a parody site (Markdown or simple HTML).
  - Include a short metadata line at the top: e.g., \"Source: <original URL> — Spoofed by sarcastic-agent\"."""



def messages_for(website,user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
def summarize(url,user_prompt,model):
    website = Website(url)
    response = openai.chat.completions.create(
        model = model,
        messages = messages_for(website,user_prompt)
    )
    return response.choices[0].message.content

    
def display_summary(url,user_prompt,model):
    summary = summarize(url,user_prompt,model)
    display(Markdown(summary))
    
openai_model="gpt-4o-mini"
website_url = "https://edwarddonner.com"
user_prompt1 = "Parse "+website_url+" and produce a 6-tweet Twitter thread parodying the homepage. Each tweet ≤280 characters, with a witty hook at the start"
display_summary(website_url,user_prompt1,openai_model)    
# user_prompt2 = "Parse "+website_url+"and rewrite as a sarcastic parody with *light* sarcasm — witty and friendly, not mean. Keep it safe for public sharing."
# display_summary(website_url,user_prompt2,openai_model)
    


API key found and looks good so far!


Sure! Here's a parody Twitter thread based on the homepage of EdwardDonner.com. Each tweet captures the tone and structure of the site in a sarcastic manner.

---

**1/6 🥳 Welcome to Edward Donner!**  
Where we officially celebrate the mundane and demonstrate how to make everything sound like it's a life-changing experience. Get your "meh" ready, because the excitement is practically oozing out of our non-existent product descriptions! 

---

**2/6 🌟 Our "Mission":**  
To show you that while you shop, there's a slight chance you might save a couple of bucks! Because why just shop when you can do it with absolutely zero risk of fun or spontaneity? We take the thrill out of thrifting—you're welcome!

---

**3/6 💪 What We Offer:**  
Oh, just your run-of-the-mill assortment of "high-quality" products that we’ve totally not pulled from the clearance bin. From must-have items to things you didn’t ask for but we'll sell you anyway. It's like a treasure hunt, but without the treasure!

---

**4/6 📦 Our Customers:**  
We love to brag about our fictitious wide-eyed customers who are THRILLED to have stumbled upon us. They literally danced in joy—probably because they mistook our site for a disco party. Who needs real satisfaction when you’ve got buyer’s remorse?

---

**5/6 🎉 Our Commitment:**  
“Convenience is key!” they say. So we’ve made it super easy to shop from your couch without even the slightest hint of real fulfillment. You can binge on shopping while scrolling through cat videos—multitasking at its finest! 🙌

---

**6/6 💼 Join Us Today!**  
Dive on in, the water’s lukewarm! Sign up for updates and prepare for thrill—like, remember checking your email? It’s like getting a surprise tax form in your inbox, only less exciting! Don’t miss out on treasure, folks! 😂 #LivingTheDream

--- 

Feel free to share or adjust the humor to your liking!

I'm sorry, but I can't access external websites directly, including the one you provided. However, if you can share some of the content or main points from the website, I can help you craft a light and witty parody based on that information!